## The General Mass Balance Equation

For any quantity (mass, moles of a specific chemical, energy, momentum) within a defined **control volume** (our system, e.g., a reactor), the following relationship must hold true:

$$ \text{Accumulation} = \text{In} - \text{Out} + \text{Generation} - \text{Consumption} $$

Let's break down each term from an engineering perspective:

*   **Accumulation:** The rate of change of the quantity *inside* the control volume. Mathematically, this is almost always a time derivative: $\frac{d(\text{Quantity})}{dt}$. If this term is zero, the system is at **steady-state**.

*   **In:** The rate at which the quantity enters the control volume across its boundaries. This is often a flow rate.

*   **Out:** The rate at which the quantity leaves the control volume across its boundaries.

*   **Generation:** The rate at which the quantity is created *within* the control volume. For a mole balance, this is the rate of a chemical reaction producing the substance.

*   **Consumption:** The rate at which the quantity is destroyed *within* the control volume. For a mole balance, this is the rate of a chemical reaction consuming the substance.

## Case Study 1: A Simple Mixing Tank (No Reaction)

Let's start with the simplest case: a tank where we are mixing a salt solution. We will perform a balance on the **mass of salt** in the tank.

![Mixing Tank](https://www.sanfoundry.com/wp-content/uploads/2016/09/plc-program-control-mixing-tank-01.png)

**1. Define the System:** Our control volume is the liquid inside the tank. The quantity we are tracking is the mass of salt, $m_{salt}$.

**2. Apply the General Balance Equation Term by Term:**
*   **Accumulation:** Rate of change of salt mass in the tank. $$ \text{Accumulation} = \frac{dm_{salt}}{dt} = \frac{d(C_{salt} \cdot V)}{dt} $$ Where $C_{salt}$ is the mass concentration (kg/m³) and $V$ is the tank volume.
*   **In:** The mass flow rate of salt entering the tank. $$ \text{In} = F_{in} \cdot C_{salt, in} $$ Where $F_{in}$ is the volumetric flow rate (m³/s).
*   **Out:** The mass flow rate of salt leaving the tank. $$ \text{Out} = F_{out} \cdot C_{salt} $$ (The concentration out is the same as the concentration inside, assuming perfect mixing).
*   **Generation:** $0$. Salt is not being created.
*   **Consumption:** $0$. Salt is not being consumed.

**3. Assemble the Final Model:**
$$ \frac{d(C_{salt} V)}{dt} = F_{in} C_{salt, in} - F_{out} C_{salt} $$

In [ ]:
from scipy.integrate import solve_ivp
import numpy as np
import matplotlib.pyplot as plt

def mixing_tank_model(t, C_salt, F_in, C_salt_in, F_out, V):
    """ODE for a simple mixing tank with constant volume."""
    # If V is constant, d(C*V)/dt = V*dC/dt
    dCsalt_dt = (F_in * C_salt_in - F_out * C_salt) / V
    return dCsalt_dt

# --- Simulation ---
# Parameters
F_in, F_out = 0.5, 0.5 # m^3/s
C_salt_in = 50.0       # kg/m^3
V = 10.0               # m^3
C_salt_initial = 0.0   # Tank starts with pure water

# Solve the ODE
t_span = (0, 60) # seconds
solution = solve_ivp(mixing_tank_model, t_span, [C_salt_initial], 
                         args=(F_in, C_salt_in, F_out, V), dense_output=True)

t_plot = np.linspace(t_span[0], t_span[1], 200)
C_plot = solution.sol(t_plot)[0]

# --- Plotting ---
plt.figure(figsize=(10,6))
plt.plot(t_plot, C_plot, linewidth=3)
plt.title('Salt Concentration in a Mixing Tank Over Time', fontsize=16, weight='bold')
plt.xlabel('Time (seconds)', fontsize=12)
plt.ylabel('Salt Concentration (kg/m$^3$)', fontsize=12)
plt.axhline(y=C_salt_in, color='red', linestyle='--', label='Inlet Concentration (Steady State)')
plt.legend()
plt.grid(True)
plt.show()

print("The simulation shows the tank concentration starting at zero and asymptotically approaching the inlet concentration, which is the expected steady state.")

## Case Study 2: An Ideal Batch Reactor

Now, let's consider a different system: a sealed tank where a reaction $A \rightarrow B$ is occurring. This is a batch reactor.

**1. Define the System:** Control volume is the contents of the reactor. We will do a mole balance on reactant A. The number of moles is $N_A = C_A V$.

**2. Apply the General Balance Equation Term by Term:**
*   **Accumulation:** $$ \text{Accumulation} = \frac{dN_A}{dt} = \frac{d(C_A V)}{dt} $$ Since V is constant, this is $ V \frac{dC_A}{dt} $.
*   **In:** $0$. It's a sealed batch reactor; nothing enters.
*   **Out:** $0$. Nothing leaves.
*   **Generation:** $0$. A is a reactant.
*   **Consumption:** The rate of reaction, $r_A = kC_A$, is per unit volume. The total consumption is the rate times the volume.
    $$ \text{Consumption} = r_A V = k C_A V $$

**3. Assemble the Final Model:**
$$ V \frac{dC_A}{dt} = 0 - 0 + 0 - k C_A V $$ 
Dividing by V gives the familiar batch reactor design equation:
$$ \frac{dC_A}{dt} = -k C_A $$

## Case Study 3: The Continuous Stirred-Tank Reactor (CSTR)

This is the grand synthesis. A CSTR is a combination of the first two cases: it has flow **AND** reaction. Let's derive its model from the general balance.

**1. Define the System:** Control volume is the reactor contents. We will do a mole balance on reactant A ($N_A = C_A V$).

**2. Apply the General Balance Equation Term by Term:**
*   **Accumulation:** $ V \frac{dC_A}{dt} $ (assuming constant V).
*   **In:** $ F_{in} C_{A,in} $.
*   **Out:** $ F_{out} C_{A} $.
*   **Generation:** $0$.
*   **Consumption:** $ k C_A V $.

**3. Assemble the Final Model:**
$$ V \frac{dC_A}{dt} = F_{in} C_{A,in} - F_{out} C_{A} - k C_A V $$ 
Dividing by V gives the final dynamic model for a CSTR:
$$ \frac{dC_A}{dt} = \frac{F_{in}}{V} (C_{A,in} - C_{A}) - k C_A $$ 
(Assuming $F_{in}=F_{out}$)

**This is exactly the model we used in the Reaction Engineering notebooks! We have now proven that it comes directly from the same universal law as all the others.**

## Conclusion: One Equation to Rule Them All

This lesson is the most conceptual, but also the most powerful. If you can master the art of applying the General Mass Balance, you can derive the model for **any** chemical process.

You have learned:

✅ The five terms of the General Mass Balance equation.

✅ How to translate each conceptual term into a mathematical expression for flow, reaction, and accumulation.

✅ How to apply simplifying assumptions (steady-state, no reaction, no flow) to tailor the general equation to a specific system.

✅ That the models for vastly different-looking processes like simple mixing, batch reactions, and continuous reactions all originate from the exact same fundamental principle.

**The General Mass Balance is your first principle.** When you face a new, complex problem, start here. Define your system, write down the balance, and analyze each term. The path to the solution will always begin with this equation.